In [ ]:
#| default_exp compat

In [ ]:
#| export
from typing import Union

import pandas as pd

In [ ]:
#| export
try:
    from polars import Categorical as pl_Categorical
    from polars import DataFrame as pl_DataFrame
    from polars import Datetime as pl_Datetime
    from polars import Float64 as pl_Float64
    from polars import Series as pl_Series
    from polars import col as pl_col
    from polars import concat as pl_concat
    from polars import from_numpy as pl_from_numpy
    from polars import lit as pl_lit

    POLARS_INSTALLED = True
except ImportError:
    pl_Categorical = type(None)
    pl_DataFrame = type(None)
    pl_Datetime = type(None)
    pl_Float64 = type(None)
    pl_Series = type(None)
    pl_col = None
    pl_concat = None
    pl_from_numpy = None
    pl_lit = None

    POLARS_INSTALLED = False

try:
    import plotly  # noqa: F401

    PLOTLY_INSTALLED = True
except ImportError:
    PLOTLY_INSTALLED = False

try:
    import plotly_resampler  # noqa: F401

    PLOTLY_RESAMPLER_INSTALLED = True
except ImportError:
    PLOTLY_RESAMPLER_INSTALLED = False

DataFrame = Union[pd.DataFrame, pl_DataFrame]
Series = Union[pd.Series, pl_Series]